<a href="https://colab.research.google.com/github/junhojjangjjangman/Classification_of_text_data/blob/main/%5BNotebook_Student%5D_Module_25_(Classification_of_text_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 데이터 분류

자연어 처리의 일반적인 응용 프로그램 중 하나는 텍스트 데이터를 분류하는 것입니다. 분류는 다양한 형태로 수행할 수 있습니다: 트윗이 특정 주제와 관련이 있는지 여부를 분류할 수 있으며, 특정 리뷰가 긍정적 또는 부정적으로 인지도 분류할 수 있습니다.

오늘은 NLP를 사용하여 NLP 데이터를 분류하는 기술을 실습하겠습니다! 
먼저, 데이터를 수집, 분석 및 처리할 것입니다. 단어 가방(Bag of words)과 tf-idf 모형을 사용할 것입니다.(데이터 획득 단계에서 수행 한 활동을 기업하십니까?) 이러한 과정은 텍스트를 숫자로 변환합니다.
그 다음 우리가 만든 단어 벡터를 사용하여 기계 학습 알고리즘을 사용하여 데이터를 훈련시키고, 분류 작업을 수행하는 모델을 만듭니다.

시작해 봅시다!

첫 번째 작업에서는 트윗 데이터 모음을 사용하여 트윗이 자연 재해를 언급하는지 아니면 일반 트윗을 참조하는지 예측합니다.

먼저 필요한 라이브러리를 가져옵시다!

## 라이브러리 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

## csv 파일 열기
트윗 데이터 파일을 가지고 있습니까? 데이터가 없는 경우 모듈 24의 노트북을 참조하여 데이터 세트를 다운로드하는 방법을 확인하십시오.

In [ ]:
df_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataSet/module25/[Dataset]_Module25_disasters_social_media.csv',encoding='ISO-8859-1')

### 분류를 위한 데이터 분석
데이터 작업을 시작하기 전에 먼저 데이터의 일부 특성을 살펴보겠습니다. 모듈 24에서 데이터 분석을 통해 데이터 구조를 파악해 보았습니다.

In [ ]:
df_raw.head()

,Unnamed: 0,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,0,778243823,True,golden,156,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,1,778243824,True,golden,152,NaN,Relevant,1.0000,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN
2,2,778243825,True,golden,137,NaN,Relevant,1.0000,Relevant,NaN,NaN,"Heard about #earthquake is different cities, s...",14.0,NaN
3,3,778243826,True,golden,136,NaN,Relevant,0.9603,Relevant,NaN,NaN,"there is a forest fire at spot pond, geese are...",15.0,NaN
4,4,778243827,True,golden,138,NaN,Relevant,1.0000,Relevant,NaN,NaN,Forest fire near La Ronge Sask. Canada,16.0,NaN


head() 함수를 사용하면 데이터 세트에서 처음 몇 행을 볼 수 있습니다. 데이터 전체의 행이 확인할 수 있습니까?

### 실습: 테이터 프레임의 길이를 출력합니다.

In [ ]:
#your code here
len(df_raw)

10876

위에서 볼 수 있는 표제는 무엇입니까? 어느 표제가 중요하다고 생각합니까?

### '대상(target)'이란 무엇인가?

예측하고자 하는 영역을 '대상'이라고 합니다. 예제의 경우 트윗이 자연 재해와 관련이 있는지 또는 관련이 없는지 여부가 대상입니다. 이러한 값은 `['choose_one']`열에 반영됩니다(위에 항목 참조!). 

### '레이블(labels)'을 기억하십니까?
이 데이터 세트에서 대상의 레이블은 인적 자원에 의해 수동으로 작성되었습니다. 향후 데이터 세트에 대한 작업을 수행할 때 수동으로 레이블을 작성하거나 지원자를 찾아야 할 수 있습니다.

이 작업은 일반적으로 노력과 시간 면에서 비용이 많이 드는 작업입니다. 이 작업에 대한 적합한 직원을 찾을 수 있는 [온라인 플랫폼](https://www.mturk.com/) 도 있습니다!

### 레이블 검사

트윗들이 분류된 카테고리를 살펴봅시다. 이를 위해 해당 열 내에서 고유한 값의 수를 찾을 수 있습니다. 파이썬의 내장 함수 `set()`은 값 목록을 가져와서 총 고유 값을 출력합니다. 어떻게 작동하는지 봅시다! 

In [ ]:
set(['apple', 'orange', 'apple', 'orange', 'pears'])

{'apple', 'orange', 'pears'}

위의 코드를 이해할 수 있나요? 5개의 리스트에서 3개의 고유한 값이 있으며, 고유한 값만 출력됩니다.

### 실습: 6개의 리스트에 2개의 고유한 값이 있도록 함수를 변경하십시오!

In [ ]:
set(['apple', 'orange', 'apple', 'orange', 'orange','apple'])

{'apple', 'orange'}

아래의 코드 조각은 자연 재해에 대한 특정 트윗의 관련성 척도인 'choose_one' 열의 값을 나열합니다.

In [ ]:
df_raw.choose_one.values

array(['Relevant', 'Relevant', 'Relevant', ..., 'Relevant', 'Relevant',
       'Relevant'], dtype=object)

set() 함수를 사용하지 않고, 이 열에 얼마나 많은 고유한 값이 있는지 추측할 수 있습니까? 

### 실습: set() 함수를 사용하여 'choose_one' 열의 고유한 값을 찾아보세요.

In [ ]:
#your code here
set_choose_one = set(df_raw.choose_one.values)
set_choose_one

{"Can't Decide", 'Not Relevant', 'Relevant'}

일리가 있습니까? 트윗은 자연 재해와는 관련이 있거나 자연 재해와 관련이 없을 수 있으며, 데이터를 표시한 사람들이 트윗이 자연 재해와 관련이 있는지 여부를 결정할 수 없는 경우도 있습니다. 

현재로서는 이진 방식(관련성 있음 vs 관련성 없음)으로 예측하는 것만 관심을 갖고 있으므로, '결정할 수 없는('can't decide') 클래스는 폐기합니다. 판다 데이터 프레임에서 [기준을 사용하여 데이터 하의 설정](http://chris.friedline.net/2015-12-15-rutgers/lessons/python2/02-index-slice-subset.html) 하는 방법을 기억하십니까?

### 실습: 데이터 프레임을 부분 집합으로 나누고 choose_one 열에 'Can't Decision'이 없는 행만 사용하십시오.

In [ ]:
#your code here
df = df_raw[df_raw.choose_one != "Can't Decide"]

데이터 프레임을 출력하여 수행한 작업을 확인합니다.

In [ ]:
#your code here
df

,Unnamed: 0,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,0,778243823,True,golden,156,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,1,778243824,True,golden,152,NaN,Relevant,1.0000,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN
2,2,778243825,True,golden,137,NaN,Relevant,1.0000,Relevant,NaN,NaN,"Heard about #earthquake is different cities, s...",14.0,NaN
3,3,778243826,True,golden,136,NaN,Relevant,0.9603,Relevant,NaN,NaN,"there is a forest fire at spot pond, geese are...",15.0,NaN
4,4,778243827,True,golden,138,NaN,Relevant,1.0000,Relevant,NaN,NaN,Forest fire near La Ronge Sask. Canada,16.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10871,10871,778261105,True,golden,100,NaN,Relevant,0.7629,Relevant,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,5675678.0,NaN
10872,10872,778261106,True,golden,90,NaN,Relevant,0.9203,Relevant,NaN,NaN,Police investigating after an e-bike collided ...,4234.0,NaN
10873,10873,778261107,True,golden,102,NaN,Relevant,1.0000,Relevant,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,3242.0,NaN
10874,10874,778261108,True,golden,96,NaN,Relevant,0.8419,Relevant,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,457.0,NaN


데이터 프레임의 크기를 확인하십시오. 감소하였나요? 아니면 그대로인가요?

### 데이터 프레임의 크기 출력

In [ ]:
#your code here
len(df)

10860

이제 'text' 열과 'choose_one' 열에만 초점을 맞추겠습니다.

### 실습: 'text'와 'choose_one' 열만 사용하도록 데이터 프레임을 부분 집합 취하기

이 [문서](http://chris.friedline.net/2015-12-15-rutgers/lessons/python2/02-index-slice-subset.html)에서 레이블(열 제목)을 사용하여 데이터를 선택하는 방법을 참조하십시오!!

In [ ]:
#your code here
df = df[['text', 'choose_one']]

In [ ]:
df

,text,choose_one
0,Just happened a terrible car crash,Relevant
1,Our Deeds are the Reason of this #earthquake M...,Relevant
2,"Heard about #earthquake is different cities, s...",Relevant
3,"there is a forest fire at spot pond, geese are...",Relevant
4,Forest fire near La Ronge Sask. Canada,Relevant
...,...,...
10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,Relevant
10872,Police investigating after an e-bike collided ...,Relevant
10873,The Latest: More Homes Razed by Northern Calif...,Relevant
10874,MEG issues Hazardous Weather Outlook (HWO) htt...,Relevant


이 [문서](https://chrisalbon.com/python/data_wrangling/pandas_map_values_to_values/) 를 참조하여 관련 트윗의 경우 숫자 1로, 관련 없는 트윗의 경우 0으로 매핑합니다. 

### 실습: 'Relevant'을 1로, 'Irrelevant' 을 0으로 매핑하여 'relevance'라는 새로운 열에 넣습니다.

In [ ]:
relevance = {'Relevant':1,'Not Relevant':0}
df['relevance'] = df.choose_one.map(relevance) # 관련 값을 숫자 1에, 관련 없는 값을 숫자 0에 매핑합니다.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df

,text,choose_one,relevance
0,Just happened a terrible car crash,Relevant,1
1,Our Deeds are the Reason of this #earthquake M...,Relevant,1
2,"Heard about #earthquake is different cities, s...",Relevant,1
3,"there is a forest fire at spot pond, geese are...",Relevant,1
4,Forest fire near La Ronge Sask. Canada,Relevant,1
...,...,...,...
10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,Relevant,1
10872,Police investigating after an e-bike collided ...,Relevant,1
10873,The Latest: More Homes Razed by Northern Calif...,Relevant,1
10874,MEG issues Hazardous Weather Outlook (HWO) htt...,Relevant,1


우리가 한 일을 보세요!
- 관심 있는 칼럼만 골라서 13개였던 칼럼을 단 3개로 줄였습니다!
- '관련성'을 1로, '관련성 없음'을 0으로 매핑했습니다.

이제 텍스트 처리를 진행하고, 단어 가방 모형과 tf-idf를 진행하겠습니다!

### 토큰화

첫 번째 단계는 트윗을 정규화하고 토큰화하는 기능을 작성하는 것입니다.(모듈 24에서 수행하였습니다). 아래에 예시가 있지만, 이전에 배운 기술을 활용하여 개선할 수 있습니다. 

예를 들어, 금지어를 추가하거나 레밍(Lemming) 또는 스티밍(Stemming)으로 데이터 세트를 줄일 수 있습니다. 데이터를 더 많이 전처리 할수록 모델이 더 좋아질 수 있습니다!
  
금지어 목록에서 그 단어를 금지하도록 선택한 이유는 무엇입니까?

In [ ]:
def extract_words(sentence):
    '''This is to clean and tokenize words'''
    ignore_words = ['a', 'the', 'if', 'br', 'and', 'of', 'to', 'is']
    words = re.sub("[^\w]", " ",  sentence).split() # this replaces all special chars with ' '
    words_cleaned = [w.lower() for w in words if w not in ignore_words]
    return words_cleaned 

# 테스트해 봅시다!
test_sentence = 'Good morning, how are you today? It is a good day.'
print(extract_words(test_sentence))

['good', 'morning', 'how', 'are', 'you', 'today', 'it', 'good', 'day']


### 실습: extract_words 함수에 최소 5개 이상의 금지 단어를 추가합니다.

In [ ]:
#your code here
test_sentence = 'Good morning, how are you today? It is a good day. if a good the fuck'
print(extract_words(test_sentence))

['good', 'morning', 'how', 'are', 'you', 'today', 'it', 'good', 'day', 'good', 'fuck']


### 실습: 같은 문장에서 새로운 금지어를 시험해보세요!

In [ ]:
# 테스트해 봅시다!
test_sentence = 'Good morning, how are you today? It is a good day.'
print(extract_words(test_sentence))

['good', 'morning', 'how', 'are', 'you', 'today', 'it', 'good', 'day']


'it' 라는 단어가 여전히 존재하고 있다는 것을 알고 있습니까? 왜 그런지 아십니까?

### 실습: extract_words 함수에 소문자를 반영하는 기능 추가
방법은 [여기](https://machinelearningmastery.com/clean-text-machine-learning-python/) 를 참조하십시오. 

In [ ]:
def extract_words(sentence):
    '''This is to clean and tokenize words'''
    ignore_words = ['a', 'the', 'if', 'br', 'and', 'of', 'to', 'is', 'are', 'he', 'she', 'my', 'you', 'it','how']
    words = re.sub("[^\w]", " ",  sentence).split() # 모든 특수 문자를 ' '로 대체합니다.
    words = [word.lower() for word in words]
    words_cleaned = [w.lower() for w in words if w not in ignore_words]
    return words_cleaned 

같은 문장으로 다시 해보세요!

In [ ]:
# 테스트해 봅시다!
test_sentence = 'Good morning, how are you today? It is a good day.'
print(extract_words(test_sentence))

['good', 'morning', 'today', 'good', 'day']


좋습니다! 스티밍 또는 레밍과 같은 자체 프로세스를 파이프라인에 자유롭게 추가할 수 있습니다.

## 1. 단어 가방 모형
이제 텍스트 데이터를 전처리하는 기능이 생겼으니 텍스트 데이터를 숫자로 변환하는 작업을 진행할 수 있습니다. 이를 위한 가장 간단한 방법은 단어 가방 모형 알고리즘을 사용하는 것이다. 

단어 가방에서 각 단어가 각 트윗에 대해 나타나는 횟수를 계산하고 해당 수를 입력 데이터로 사용합니다. 이 작업은 다음 단계를 통해 수행됩니다:
1. 말뭉치에 나타나는 모든 단어의 어휘를 만듭니다(말뭉치는 모든 텍스트 데이터의 모음입니다. 즉 모든 트윗).
2. 그 어휘를 벡터로 전환합니다. 예를들어 말뭉치에 500개의 고유한 단어가 있는 경우, 어휘의 벡터의 크기는 500이며 각 위치는 말뭉치의 단어에 해당합니다.
3. 각 문서 (트윗)에 대해 모든 단어가 나타나는 횟수를 계산하고 해당 숫자를 벡터에 추가합니다.  이렇게 하면 각 문서에 고유한 길이 500 벡터가 생성되어 문서에 나타나는 모든 단어를 나타냅니다.

### 예시 
두 개의 문서로 구성된 말뭉치를 고려합니다: 
1. 'I love NLP', 
2. 'I love machine learning'. 

### 어휘 
어휘는 다음과 같은 단어로 구성된 길이 5의 벡터가 됩니다: 

'I', 'love', 'NLP', 'machine', 'learning'.  

#### 벡터
첫 번째 문장 (1 번)에 대한 벡터는 다음과 같습니다.: 
[1, 1, 1, 0, 0] 'I', 'love', 'NLP'가 포함되어 있지만 'machine', 'learning'은 포함되어 있지 않기 때문입니다..  

2에 대한 벡터를 구성할 수 있습니까?

1과 2의 벡터를 결합하면, 단어 가방은 첫 번째 행에 벡터 1이 있고, 두 번째 행에 벡터 2가 있는 배열이 됩니다.

이제 이 알고리즘을 구현하겠습니다!

### 단어 가방 만들기

먼저 데이터 세트에 각 단어가 얼마나 자주 나타나는지 알고 싶습니다. 

우리는 이것을 사전의 형태로 나타낼 수 있는데, 사전의 형식은 {'word':frequency}이며, 여기서 각 키는 'word'이고 frequency는 단어가 데이터 세트에 나타나는 횟수입니다.

사전에 대해 더 알고 싶으시면 [파이써 딕셔너리](https://www.w3schools.com/python/python_dictionaries.asp) 를 참조하십시오.  

### 해시 맵

이 사전을 해시 맵이라고 하며 문서의 각 토큰을 반복하여 점진적으로 구축 할 수 있습니다.

해시 맵에서 토큰을 찾을 수 없는 경우 토큰을 해시 맵에 추가하고 빈도를 1로 설정합니다. 토큰이 이미 있으면 빈도를 1씩 증가시킵니다. 
  
우리는 두 가지 기능에서 이 작업을 수행 할 것입니다. 
1. 먼저 hash_map이라는 사전과 트윗의 토큰을 가져 와서 토큰의 각 단어로 hash_map을 업데이트하는 map_book이라는 함수를 만듭니다. 
2. 다음으로, 모든 트윗을 반복할 수 있는 함수(make_hash_map이라고 부를 수 있음)를 만들고, hash_map을 업데이트하는 첫 번째 함수를 호출합니다.
  
*힌트: 
`for word in tokens:` 를 사용하여 토큰을 반복하여 사용할 수 있습니다.  
`if word in hash_map:`을 사용하여 단어가 해시 맵에 존재하는지 확인할 수 있습니다.  
`hash_map[word]`을 사용하여 해시 맵의 계수를 평가할 수 있습니다. 이 단어를 1씩 증가시켜 카운트를 늘립니다.

이것은 map_book 함수입니다. 이해할 수 있습니까?

In [ ]:
# 단어의 빈도를 계산합니다
def map_book(hash_map, tokens):
    if tokens is not None:
        for word in tokens:
            # 단어가 존재합니까?
            if word in hash_map:
                hash_map[word] = hash_map[word] + 1
            else:
                hash_map[word] = 1

        return hash_map
    else:
        return None

이것은 make_hash_map 함수입니다. 이해할 수 있겠습니까?

In [ ]:
def make_hash_map(df):
    hash_map = {}
    for index, row in df.iterrows():
        hash_map = map_book(hash_map, extract_words(row['text']))
    return hash_map

### 사전을 다시 정의합니다.

모든 트윗의 모든 단어를 사용하여 단어 가방을 구성할 수 있지만 데이터의 양이 많이 때문에 컴퓨터에 많은 부담이 될 수 있습니다. 좋은 해결책은 가장 흔한 단어 몇 백 개 혹은 수천 개만 가져가는 것입니다. 우리는 우리의 사전을 500개의 가장 인기 있는 토큰들로만 구성되도록 재정립할 것입니다.
  
어떻게 이런 일을 할 수 있을까요? 각 토큰이 있는 사전인 해시 맵과 토큰이 값으로 표시된 횟수를 방금 작성했다는 점을 기억하십시오. hash_map과 최대 어휘를 사용하는 frequent_vocab이라는 함수를 작성하고 최대 어휘에 정의 된대로 가장 인기있는 토큰 목록을 반환합니다(지금은 500으로 설정).

이것은 frequest_vocab 함수입니다. 이해할 수 있겠습니까?

In [ ]:
# frequent_vocab 함수를 다음과 같은 입력으로 정의하십시오 : word_freq 및 max_features
def frequent_vocab(word_freq, max_features): 
    counter = 0  # 값 0으로 카운터를 초기화하십시오
    vocab = []   # Vocab이라는 빈 목록을 만듭니다
    # 단어를 빈도수가 낮은 순서로 사전에 나열합니다
    for key, value in sorted(word_freq.items(), key=lambda item: (item[1], item[0]), reverse=True): 
       # 상위(max_features) 단어 수를 얻기 위한 루프 함수
        if counter<max_features: 
            vocab.append(key)
            counter+=1
        else: break
    return vocab

### 실험! 위의 함수를 (reverse = False)로 변경하면 어떻게 됩니까?

In [ ]:
hash_map = make_hash_map(df) #토큰 화 된 데이터 세트에서 해시 맵 (단어 및 빈도) 생성

vocab=frequent_vocab(hash_map, 500)
hash_map

### 실험! max_feature를 100으로 변경하면 어떻게 됩니까?
나중에 500 개 이상을 되돌려 놓는 것을 잊지 마십시오.

### 마침내 우리는 단어 가방을 구축합니다

In [ ]:
# d다음과 같은 입력으로 함수 bagofwords를 정의: sentence, words
def bagofwords(sentence, words):
    sentence_words = extract_words(sentence) # 문장/트윗을 토큰화하고 변수 sentence_words에 할당
    # 빈도 단어 수
    bag = np.zeros(len(words)) # 크기가 len(words)이고 0으로 구성된 NumPy 배열 생성
    # 트윗에 토큰이 있을 때 데이터를 반복하고 1의 값을 추가
    for sw in sentence_words:
        for i,word in enumerate(words):
            if word == sw: 
                bag[i] += 1
                
    return np.array(bag) # 하나의 트윗에 대한 단어 가방 반환

### 실습: 작성된 텍스트 데이터를 사용하여 기능을 테스트합니다.
위의 단어 목록을 보고 샘플 텍스트에 추가할 단어를 확인하십시오.

In [ ]:
text = 't co http  in for'
bagofwords(text, vocab)

array([1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

단어 가방의 한 행을 보세요!

이제 전체 데이터 세트를 통해 이 함수를 반복하려고 합니다. 어떻게 하는지는 아래를 참조하십시오.

In [ ]:
# 단어 가방을 포함하도록 지정된 차원이있는 숫자 배열을 설정합니다.
n_words = len(vocab)
n_docs = len(df)
bag_o = np.zeros([n_docs,n_words])
# 루프 함수를 사용하여 각 트윗에 대해 새 행을 추가합니다.
for ii in range(n_docs): 
    # 이전 함수 'bagofwords'를 호출합니다. 입력을 참조하십시오 : sentence, words
    bag_o[ii,:] = bagofwords(df['text'].iloc[ii], vocab) 

이제 numpy 배열의 [차원](https://stackoverflow.com/questions/14847457/how-do-i-find-the-length-or-dimensions-size-of-a-numpy-matrix-in-python) 을 알아보십시오. 여러분에게 의미가 있습니까?

In [ ]:
#your code here
bag_o.shape

(10860, 500)

## 2. 전체 빈도, 역 문서 빈도 찾기

여기서 우리는 문장/트윗에서 가장 의미있는 단어를 사용하고자합니다 가장 자주 사용되는 단어가 중요하다고 생각하는 것이 의미가 있습니까?

우리는 먼저 우리의 단어 가방 안에 있는 단어들을 살펴봅니다. 가장 많이 사용하는 단어 20개를 출력합니다.

힌트: hash_map은 각 단어를 키로 사용하는 모든 워드와 해당 빈도를 dict{word: frequency} 값으로 표시한 사전입니다. 이 가장 일반적인 단어에 대해 무엇을 알 수 있습니까?

자세한 내용은 '주요 기능' 아래의 [이 문서](https://docs.python.org/3/howto/sorting.html) 를 참조하십시오. 개체의 인덱스를 키로 사용합니다.

In [ ]:
#your code here
sorted(hash_map.items(), key=lambda item: item[1], reverse=True)[:20]

[('t', 7442),
 ('co', 6799),
 ('http', 6153),
 ('in', 2805),
 ('i', 2486),
 ('ã', 1633),
 ('s', 1272),
 ('for', 1243),
 ('on', 1236),
 ('that', 844),
 ('with', 798),
 ('by', 768),
 ('at', 745),
 ('this', 702),
 ('https', 618),
 ('from', 613),
 ('be', 587),
 ('was', 554),
 ('â', 535),
 ('m', 531)]

Key=Lambda 란 무엇입니까? 자세한 내용은 [이 기사](https://stackoverflow.com/questions/13669252/what-is-key-lambda/13669294) 를 참조하십시오.

위의 상위 20개 단어를 참조하십시오. 무엇을 알 수 있습니까?


### 특성 선택
가장 일반적인 20 단어는 트윗에 대한 정보가 거의 없습니다. 그것들은 모든 텍스트에서 자주 발견되는 일반적인 단어들뿐만 아니라 트위터 URL의 잔해입니다. 우리는 그것들을 '중요한 특성'이라고 보기 어렵습니다. 그렇다면 모델을 개선하기 위해서는 가장 빈번한 단어만 보는 것이 아니라 더 많은 것을 해야 할 것 같습니다.

일부 문서에는 자주 등장하지만 모든 문서에는 없는 단어들을 찾아봐야 할 것 같습니다. 왜 이것이 합리적이라고 생각합니까?

이것은 '총 빈도 역문서 빈도'(total frequency inverse document frequency, 약칭 tf-idf)로 알려진 알고리즘의 기본 개념입니다.

TFIDF 수식은 다음과 같습니다: 
$$w_{i,j}=tf_{i,j}*log(\frac{N}{df_i})$$  
이 공식에서 $ i $는 단어 인덱서이고 $ j $는 문서 인덱서입니다.
단어 가방에서 각 행은 문서인 반면, 각 열은 해당 문서에 있는 단어의 빈도입니다. 이는 이미 tfidf($tf_{i,j}$)의 '항 빈도' 부분입니다.

### 역문서 빈도

이제 다음과 같은 방식으로 이해할 수 있는 역 문서 빈도를 계산하려고합니다: 각 단어에 대해 나타나는 문서의 수를 계산한 다음 해당 숫자의 역 로그를 가져옵니다.

idf 벡터를 두 부분으로 구성합니다.
1. 먼저 각 단어에 대한 단어 빈도를 만듭니다. 
2. 다음 문서(N)를 단어 빈도로 나누고 결과 로그를 취합니다.

획득 단계에서 우리가 한 일을 기억하십니까?

In [ ]:
# 트윗 수(numdocs)와 토큰/워드 수(numwords)를 나타내는 변수 2개 초기화
numdocs, numwords = np.shape(bag_o)

# 위와 같이 TFIDF 수식으로 변경
N = numdocs
word_frequency = np.empty(numwords)

# 단어가 나타나는 문서 수를 계산
for word in range(numwords):
    word_frequency[word]=np.sum((bag_o[:,word]>0)) 

idf = np.log(N/word_frequency)

In [ ]:
idf.shape

(500,)

이제 단어 가방(용어 빈도)을 idf와 함께 입력함으로써 tfidf를 완료하겠습니다.

In [ ]:
# 초기화 tfidf 배열
tfidf = np.empty([numdocs, numwords])

# 트윗에서 반복, 용어 빈도(단어 가방으로 표시)를 idf로 곱합니다.
for doc in range(numdocs):
    tfidf[doc, :]=bag_o[doc, :]*idf

In [ ]:
tfidf.shape

(10860, 500)

In [ ]:
print (tfidf)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.56037575 0.64356806 0.74842242 ... 0.         0.         0.        ]
 [0.56037575 0.64356806 0.74842242 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


tfidf 배열에 대해 어떻게 설명하시겠습니까? 그것은 10860개의 트윗에서 500개의 토큰 각각의 tfidf 값으로 만들어집니다.

## 3. 기계 학습으로 모델을 훈련시킵니다.
드디어 TFIDF 배열이 생겼으니 여러분의 모델을 훈련시키고 예측을 할 때입니다! 우리는 많은 기계 학습 모델을 제공하는 scikit learn 라이브러리를 사용할 것입니다. 

기계 학습이란 무엇인지 기억하십니까? 시스템이 명시적으로 프로그래밍되지 않아도 자동으로 학습 할 수있는 AI의 응용 프로그램입니다.

이제 우리는 지도 학습을 사용할 것입니다. 이것이 무엇인지 기억하시나요?
이것은 훈련 세트가 주어지면 특정 시스템을 예측하는 모델을 만들 수 있게 해주는 학습 유형입니다. 
우리는 텍스트가 재난에 대한 뉴스와 관련이 있는지 여부를 예측하려고 합니다. 이미 트위터에서 텍스트 데이터를 다운로드했으며 데이터에 여러 레이블을 붙였습니다.'relevant', 'not relevant', 'can't decide' 등이 있으며, 이러한 데이터는 우리의 모델을 훈련시키는데 사용될 것입니다. 어떻게 할 수 있는지 알아보겠습니다.

먼저 이 작업을 수행하는 데 필요한 라이브러리를 다운로드합니다. scikit learn 라이브러리에는 기계 학습 문제에 사용되는 수많은 유용한 기능이 포함되어 있습니다.

## 라이브러리 가져오기

In [ ]:
from sklearn.linear_model import LogisticRegression #로지스틱 회귀 모형 가져오기
from sklearn.model_selection import train_test_split # 데이터를 훈련 및 테스트 세트로 분할
from sklearn.model_selection import GridSearchCV # 모델의 가장 적합한 매개 변수를 찾기 위해

### 1단계. 데이터를 교육 및 테스트 세트로 분할

모델을 교육하기 전에 데이터 세트를 2개로 분할합니다: 교육 세트, 테스트 세트

우리는 교육 세트에서 모델을 훈련시킨 다음 테스트 세트로 훈련 단계에서 생성 된 모델을 테스트합니다. 이는 테스트가 모델에 '보이지 않는'/처리되지 않은 데이터 세트에서 수행되는지 확인하기 위함입니다.

분할의 좋은 시작점은 훈련 세트에 데이터의 80%를, 테스트 세트에 데이터의 20%를 분할하는 것입니다.

지금 해봅시다!

In [ ]:
# X_all과 y_all을 교육 및 테스트 세트로 분할
X_train,X_test,y_train,y_test = train_test_split(tfidf,df['relevance'].values,shuffle=True)

위의 코드에서 무슨 일이 일어나고 있는지 설명해 주시겠습니까? train_test_split 함수를 사용하여 tfidf 배열과 트윗의 'relevance' 값을 포함하는 초기 데이터 프레임의 일부를 분할하고 있습니다. [Shuffle=True](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 는 무엇을 의미합니까? 

작업 중인 데이터 세트에 대해 자세히 알아보겠습니다. 아래 tfidf 및 df['relevance'] 를 출력하여 확인하십시오. 한편, [.value](https://www.geeksforgeeks.org/python-pandas-dataframe-values/) 가 무엇인지 알아보세요. 

In [ ]:
#your code here아래 tfidf 및 df['relevance'] 
print(tfidf)
print(df['relevance'])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.56037575 0.64356806 0.74842242 ... 0.         0.         0.        ]
 [0.56037575 0.64356806 0.74842242 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
0        1
1        1
2        1
3        1
4        1
        ..
10871    1
10872    1
10873    1
10874    1
10875    1
Name: relevance, Length: 10860, dtype: int64


좋습니다! 데이터 세트를 이미 분할했으므로 현재 어떤 데이터가 있는지 살펴보겠습니다. X_train, X_test, y_train 및 y_test 모양을 출력합니다!

In [ ]:
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(8145, 500)
(2715, 500)
(8145,)
(2715,)


x_train 및 x_test를 비교하고 y_train 및 y_test를 비교하십시오. 모양의 차이에 대해 무엇을 알 수 있습니까?

In [ ]:
#your answer here

### 2단계. 모델 인스턴스(instance) 작성
데이터를 분할한 후 모델의 인스턴스(예: 모델을 초기화하기만 하면 됩니다)를 만듭니다. 이 작업에서는 데이터를 분류할 때 유용한 로지스틱 회귀 분석기를 사용합니다. 자세한 내용은 [여기](https://towardsdatascience.com/understanding-logistic-regression-9b02c2aec102) 를 읽어보세요!

In [ ]:
# 모델 인스턴스(instance) 작성
logreg = LogisticRegression(solver = 'liblinear')

### 3단계. 데이터에 대한 모델 훈련, 데이터로부터 배운 정보 저장

In [ ]:
# 학습 세트에 모델 적합
logreg.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

지금은 위에 표시된 매개 변수를 이해하는 것에 대해 걱정하지 마십시오. 관련된 모든 매개변수를 모를 경우에도 로지스틱 회귀 분석기를 사용하여 프로젝트를 만들 수 있습니다.

## Step 4. 모델을 사용하여 테스트 데이터를 기반으로 관련성 예측

In [ ]:
y_pred=logreg.predict(X_test)
df_pred = pd.DataFrame(columns=['Actual', 'Predict'])
df_pred['Actual'] = y_test
df_pred['Predict'] = y_pred
df_pred.head()

,Actual,Predict
0,1,1
1,0,0
2,1,1
3,1,1
4,0,0


y_pred는 무엇을 의미합니까?

이제 모델의 정확도 값을 찾아 모델 성능을 측정하겠습니다. 정확도는 다음과 같이 정의됩니다:

올바른 예측의 분율 = 정확한 예측 / 총 데이터 포인트 수

In [ ]:
# 스코어 방법을 사용하여 모델의 정확성을 얻습니다
score = logreg.score(X_test, y_test)
print(score)

print('Accuracy of logistic regression classifier on test set: {:.3f}'.format(score))

0.7727440147329651
Accuracy of logistic regression classifier on test set: 0.773


대단합니다! 우리는 데이터를 수집하고, 처리하고, 교육 및 테스트 데이터로 나누고, 모델을 교육하고, 모델의 성능을 평가했습니다. 다음으로 이 모든 작업을 한 번에 수행할 수 있는 함수를 정의하겠습니다!

### 모델 훈련 파이프라인

이 기능은 다음과 같이 구성됩니다:
1. 훈련되지 않은 모델, tfidf 배열 및 훈련 대상의 값을 가져온다.
2. 무작위로 두 개를 나누어 학습과 테스트 세트를 만든다.
3. 학습 세트에 모델을 맞춘다.
4. 테스트 세트에 정확도 점수를 출력한다.
5. 훈련된 모델을 반환한다.

In [ ]:
def classify(rf, X_all, y_all): # 훈련되지 않은 모델, tfidf 배열 및 훈련 대상의 값을 가져온다.
    X_train,X_test,y_train,y_test = train_test_split(X_all,y_all,shuffle=True) # 무작위로 두 개를 나누어 학습과 테스트 세트를 만든다.
    logreg.fit(X_train,y_train) # 학습 세트에 모델을 맞춘다.
    print(rf.score(X_test,y_test)) # 테스트 세트에 정확도 점수를 출력한다.
    return logreg # 훈련된 모델을 반환한다.

이제 데이터 세트에 함수를 적용할 수 있습니다.

In [ ]:
logreg = LogisticRegression(solver = 'newton-cg')
#logreg = LogisticRegression()

X_all = tfidf
y_all = df['relevance'].values
logreg = classify(logreg, X_all, y_all)

0.7679558011049724


### 매개 변수 조정(옵션)
이 권한을 구현 한 경우 적어도 0.75의 테스트 점수가 있어야합니다. 

하이퍼 매개 변수를 조정하여 이 점수를 개선해 보겠습니다. 먼저 로지스틱 회귀 분석기의 매개 변수를 살펴보겠습니다. 매개 변수에 대한 자세한 내용은 [여기](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) 를 참조하십시오. 다시 말하지만, 만약 여러분이 지금 이 변수들을 이해하지 못한다고 스스로에게 스트레스를 주지 마세요. 그러한 결과는 더 많은 연습과 읽기를 통해 얻어질 수 있습니다.

In [ ]:
logreg

LogisticRegression(solver='newton-cg')

매개 변수는 Scikit-Learn의 GridSearchCV를 사용하여 자동으로 조정할 수 있습니다. 이 함수는 테스트할 값이 있는 매개변수 사전뿐만 아니라 모델을 사용하며 각 매개변수 조합을 테스트하여 최상의 스코어에 대한 최적 조합을 찾습니다. [여기](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV.) 에서 자세히 알아보세요.

In [ ]:
# 여기서 하이퍼 파라미터를 정의하십시오.
parameters = {'C':[0.001, 0.01, 0.1, 1, 10], 'tol':[0.0001, 0.001, 0.01], 'max_iter':[100, 1000]}

In [ ]:
clf = GridSearchCV(logreg, parameters, cv=3, return_train_score=True)

In [ ]:
clf.fit(X_all, y_all)

KeyboardInterrupt: ignored

In [ ]:
# clf.cv_results_를 사용하여 초기 결과를 볼 수 있다.
print(clf.best_params_, clf.best_score_)

정확도 결과를 보십시오. 떨어지는 것을 알아차리셨나요? 왜 이것이 그렇다고 생각합니까?

모델을 무작위로 분할하기 때문에 정확도가 다를 수 있습니다. 우리가 할 수 있는 것은 모델 피팅을 여러 번 반복하고 평균적인 정확도 결과를 얻는 것입니다.

## 4. 파이프라인 구축
이 시점에서 트윗을 예측할 수 있도록 훈련되고 최적화된 모델이 준비되어 있습니다. 이제 이 모든 것을 하나로 묶어 예측을 위한 파이프라인을 구축합니다.

이 기능은 다음과 같습니다:
1. 트윗을 문자열 형식으로 취한다.
2. 해당 트윗이 관련이 있는지 여부에 대한 예측을 출력한다.

In [ ]:
def twitter_predictor(tweet):
    # your code here
    word_vector = bagofwords(tweet, vocab) # 단어 가방 변수를 설정합니다.bagofwords 함수를 기억하십니까?
    word_tfidf = word_vector*idf # tfidf값 찾기
    prediction = logreg.predict(word_tfidf.reshape(1, -1)) # 트윗이 자연재해와 관련이 있는지 없는지 예측
    results = {1:'Relevant', 0:'Not Relevant'} # 잠재적인 결과를 포함하는 집합을 만듭니다."Relevant" 및 "Not relevant" 태그를 변경할 수 있습니다.
    print(results[int(prediction)])

In [ ]:
tweet1 = 'When the earthquake happened (Nepal) we were the last intl team still there; in a way we were 1st responders'
tweet2 = 'NLP is fun and I learnt so much today.'
twitter_predictor(tweet1)
twitter_predictor(tweet2)

### 실습: 트윗을 직접 작성하고 모델이 트윗을 분류할 수 있는지 확인해 보십시오!

In [ ]:
# your code here

모델이 올바른 결과를 제공할 수 있습니까? 

모델 성능을 향상시키기 위해 무엇을 할 수 있다고 생각하십니까?

이전에 상위 500 개 토큰 만 사용한다는 것을 기억하십니까? 우리가 더 많은 토큰을 선택하거나 더 적은 토큰을 선택하면 어떻게 될 것이라고 생각하십니까?

축하합니다! 이제 자신만의 기계 학습 NLP 모델을 구축했습니다.
  
# 5. NLP 분류 과제!
TFIDF가 자연어 데이터 분류를 하기 위해 정규화된 단어 가방을 사용하는 기본 사항을 배웠으니 이제는 여러분의 기술을 테스트할 때입니다!

## 감정 분석
자연 텍스트 분류의 중요한 응용은 _감정 분석(sentiment analysis)_에 있습니다. 감정 분석은 특정 주제에 대한 작가의 태도를 확인하기 위해 텍스트 조각에서 의견을 분류하는 과정입니다.
  
이 도전과제에서는 [imdb](https://www.imdb.com/)의 영화 리뷰를 분류합니다. 데이터는 이미 두 개의 .pkl 파일로 저장되었습니다(현재는 파이썬을 사용하여 읽을 수있는 파일 형식으로 이해하십시오). 하나는 훈련 데이터용이고 하나는 테스트 데이터용 입니다.

You will have to process and train your model on the train dataset of movie reviews `df_raw.pkl`, and then report the accuracy of your model on the test move reviews `df_raw_test.pkl`.  
영화 리뷰 `df_raw.pkl`의 훈련 데이터 세트에서 모델을 교육한 다음, 영화 리뷰 `df_raw_test.pkl` 테스트 데이터 세트를 이용하여 모델의 정확도를 확인해야 합니다.

리뷰가 긍정적인 정서를 가지고 있는지 부정적인 정서를 가지고 있는지 예측하게 될 것입니다. 긍정적인 정서는 1로 분류되고, 부정적인 정서는 0으로 분류됩니다.
  
이 세그먼트에서는 sklearn에서 제공하는 몇 가지 새로운 기능을 사용합니다.
1. 이전에 구축한 함수로 TFIDF를 사용하여 단어 가방을 생성하고 조건을 조정할 수 있습니다. 
2. 대안적으로, [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) 기능을 이용하여 단어 가방을 만들 수 있습니다. `max_features=5000` 인수를 사용하여 가장 일반적인 상위 5000개의 단어만 선택하여 사용합니다.
3. TFIDF로 단어 가방을 변환하는 데 도움이 되는 [`TfidfTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer) 를 사용할 수도 있습니다.
  
이 데이터 프레임에 대한 훈련 및 테스트 데이터 세트는 이미 가져왔습니다. 이전에 배운 기술과 모듈 24에서 배운 기술을 사용하여, 데이터 전처리, 벡터화(단어 가방), 변화(TFIDF 사용), 적합성 등을 적용하여 영화 리뷰의 감정을 예측해야 합니다.

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer # 이 기능은 단어 가방을 만들 수 있도록 도와줍니다.
from sklearn.feature_extraction.text import TfidfTransformer # 이 기능은 단어 가방을 자동으로 정규화합니다.
df_raw = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/DataSet/module25/[Dataset]_Module25_df_raw.pkl')
df_raw_test = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/DataSet/module25/[Dataset]_Module25_df_raw_test.pkl')


## 7번째 줄과 8번째 줄에 오류가 있는 경우, "imdb" 폴더에서 파일을 복사하여 노트북과 같은 위치에 붙여 넣으십시오.

시작하려면 테스트 데이터 세트의 샘플을 출력해 보십시오. 열의 이름은 무엇입니까?

In [ ]:
# your code here
df_raw_test

Now, process your text using the `CountVectorizer` to create your bag of words. 
이제 `CountVectorizer`를 사용하여 텍스트를 처리하여 단어 가방을 만듭니다. `CountVectorizer()`클래스를 만들고 텍스트와 함께 `.fit_transform()` 메서드를 사용하여 단어 모음을 만들 수 있습니다.

In [ ]:
vectorizer = CountVectorizer(analyzer = "word", strip_accents=None, tokenizer = None, \
                             preprocessor = None, stop_words = None, max_features = 5000) 
train_data_features = vectorizer.fit_transform(df_raw['text'])
test_data_features = vectorizer.transform(df_raw_test['text'])

이제 `TfidfTransformer`를 사용하여 단어 가방을 정규화합니다. 사용 방법은 동일합니다.클래스를 만들고 단어 가방과 함께 `.fit_transform()` 메서드를 인수로 사용하여 TFIDF를 만듭니다.

In [ ]:
tfidfier = TfidfTransformer()
tfidf = tfidfier.fit_transform(train_data_features)
tfidf_test = tfidfier.transform(test_data_features)

이제 변환된 단어 가방을 사용하여 이전과 같이 모델을 훈련하고 테스트합니다.

In [ ]:
X_all = tfidf.toarray()
y_all = df_raw['positive'].values
X_test = tfidf_test.toarray()
y_test = df_raw_test['positive'].values
def classify():
    rf = LogisticRegression()
    rf.fit(X_all,y_all)
    print(rf.score(X_test,y_test))
    return rf
classify()

하이퍼파라미터를 조정하지 않고도 80% 이상의 정확도를 가지고 있어야 합니다. 가능한 테스트 세트에서 최상의 정확도를 얻으십시오. 이 데이터 세트는 자연어 처리의 특징 중 하나이며 많은 데이터 과학자와 엔지니어들의 진입점이다. 다른 사람들이 개발한 더 많은 [솔류션](https://www.kaggle.com/c/word2vec-nlp-tutorial) 을 볼 수 있습니다!

## 다음으로는 자신의 리뷰를 입력하는 함수를 만들어 자신의 문장이 긍정적인지 부정적인지를 예측하는 모델을 만들 수 있습니다!

# 축하합니다!
자연 언어 텍스트 분류기를 만드는 방법을 배웠습니다!